In [1]:
import numpy as np
from PIL import Image
import os
import shutil
from collections import defaultdict
import random
import json
random.seed(42)


def resize_pad(image, size):
    """
    Resize and pad an image to a square image of size size
    """
    # Resize
    width, height = image.size
    if width > height:
        image = image.resize((size, int(height * size / width)))
    else:
        image = image.resize((int(width * size / height), size))

    #make the image black and white, not grayscale
    image = image.convert('L')
    # Pad
    new_image = Image.new('L', (size, size), 255)
    new_image.paste(image, ((size - image.size[0]) // 2, (size - image.size[1]) // 2))
    return new_image

In [2]:

SRC_DIR = "base_for_eval"
DST_DIR = "EVAL"

for filename in os.listdir(DST_DIR):
    file_path = os.path.join(DST_DIR, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)

    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [3]:
global_hashes = set()


def make_combinations(folder1, folder2, dst_dir, priority = False):

    if "/real/" in folder1:
        folder1_type = "real"
    else:
        folder1_type = "impossible"
    
    if "/real/" in folder2:
        folder2_type = "real"
    else:
        folder2_type = "impossible"

    i = len(os.listdir(dst_dir))
    for filename1 in os.listdir(folder1):
        for filename2 in os.listdir(folder2):
            file_path1 = os.path.join(folder1, filename1)
            file_path2 = os.path.join(folder2, filename2)
            
            if file_path1 == file_path2:
                continue
            
            img1 = Image.open(file_path1)
            img2 = Image.open(file_path2)
            img1 = resize_pad(img1, 256)
            img2 = resize_pad(img2, 256)
            
            new_img = Image.new('L', (512, 256))
            new_img.paste(img1, (0, 0))
            new_img.paste(img2, (256, 0))

            h = hash(new_img.tobytes())
            if h in global_hashes:
                continue
            global_hashes.add(h)

            if priority:
                new_img.save(os.path.join(dst_dir, f"{i:04d}_{folder1_type}_{folder2_type}_p.png"))
            else:
                new_img.save(os.path.join(dst_dir, f"{i:04d}_{folder1_type}_{folder2_type}.png"))
            i += 1

            new_img = Image.new('L', (512, 256))
            new_img.paste(img2, (0, 0))
            new_img.paste(img1, (256, 0))

            h = hash(new_img.tobytes())
            if h in global_hashes:
                continue
            global_hashes.add(h)
            
            if priority:
                new_img.save(os.path.join(dst_dir, f"{i:04d}_{folder2_type}_{folder1_type}_p.png"))
            else:
                new_img.save(os.path.join(dst_dir, f"{i:04d}_{folder2_type}_{folder1_type}.png"))
            i += 1


In [4]:
types = [1,2,3,4,5]

for t1 in types:
    for t2 in types:
        make_combinations(f"base_for_eval/real/{t1}", f"base_for_eval/impossible/{t2}", DST_DIR, t1==t2)

len(os.listdir(DST_DIR))


1156

In [5]:
for t1 in types:
    for t2 in types:
        make_combinations(f"base_for_eval/real/{t1}", f"base_for_eval/real/{t2}", DST_DIR, t1==t2)
        
len(os.listdir(DST_DIR))


1428

In [6]:
for t1 in types:
    for t2 in types:
        make_combinations(f"base_for_eval/impossible/{t1}", f"base_for_eval/impossible/{t2}", DST_DIR, t1==t2)
        
len(os.listdir(DST_DIR))

2550

In [7]:

d = defaultdict(list)

for filename in os.listdir(DST_DIR):
    x = filename.split(".")[0][5:]

    d[x].append(filename)



In [8]:
final_pick = []

for k in d:
    print(k, len(d[k]))

real_impossible_p 113
impossible_real_p 113
real_impossible 465
impossible_real 465
real_real_p 50
real_real 222
impossible_impossible_p 204
impossible_impossible 918


In [9]:


TARGET = 250
real_impossible = []
impossible_real = []
real_real = []
impossible_impossible = []

if len(d["real_real_p"]) > TARGET:
    real_real = random.sample(d["real_real_p"], TARGET)
else:
    real_real = d["real_real_p"]
    real_real += random.sample(d["real_real"], TARGET - len(real_real))

if len(d["real_impossible_p"]) > TARGET:
    real_impossible = random.sample(d["real_impossible_p"], TARGET)
else:
    real_impossible = d["real_impossible_p"]
    real_impossible += random.sample(d["real_impossible"], TARGET - len(real_impossible))

if len(d["impossible_real_p"]) > TARGET:
    impossible_real = random.sample(d["impossible_real_p"], TARGET)
else:
    impossible_real = d["impossible_real_p"]
    impossible_real += random.sample(d["impossible_real"], TARGET - len(impossible_real))

if len(d["impossible_impossible_p"]) > TARGET:
    impossible_impossible = random.sample(d["impossible_impossible_p"], TARGET)
else:
    impossible_impossible = d["impossible_impossible_p"]
    impossible_impossible += random.sample(d["impossible_impossible"], TARGET - len(impossible_impossible))

In [10]:
len(real_impossible), len(impossible_real), len(real_real), len(impossible_impossible)

final_pick = real_real + real_impossible + impossible_real + impossible_impossible

In [11]:
import random
random.seed(42)
random.shuffle(final_pick)

In [12]:
question = "Which object is geometrically impossible?"
options = ["The left object", "The right object", "Both objects", "Neither object"]
# options = ["Neither object", "The right object", "The left object", "Both objects"]
# [
#   {
#     "question": "Are the two birds the same color?",
#     "options": [
#         "no",
#         "yes",
#         "cannot tell from the image"
#     ],
#     "answer": "yes",
#     "image": "110.jpg",
#     "category": "color"
#   }
# ]

for_json = []
for i, filename in enumerate(final_pick):
    d = {}
    d["question"] = question
    d["options"] = options

    if "real_real" in filename:
        d["answer"] = "Neither object"
    elif "real_impossible" in filename:
        d["answer"] = "The right object"
    elif "impossible_real" in filename:
        d["answer"] = "The left object"
    elif "impossible_impossible" in filename:
        d["answer"] = "Both objects"

    d["image"] = filename
    d["category"] = "soft_localization"
    for_json.append(d)



In [13]:
with open('eval_labels.json', 'w') as f:
    json.dump(for_json, f, indent=2)